# 🚀 LOGISTICS PRODUCTION — Batch Processing

```
╔══════════════════════════════════════════════════════════════════════════════╗
║              FRÉGATE 02_LOGISTICS — PRODUCTION PIPELINE                      ║
║                   Batch • Automatisation • Rapport                           ║
╚══════════════════════════════════════════════════════════════════════════════╝
```

**Objectif**: Traiter en batch tous les avatars animés et leur attacher les props.

---

## 📦 Cellule 1: Montage Drive + Config

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Configuration
DRIVE_ROOT = "/content/drive/MyDrive/DRIVE_EXODUS_V2"
UNIT_ROOT = f"{DRIVE_ROOT}/02_LOGISTICS_DEPOT"
CODEBASE = f"{UNIT_ROOT}/CODEBASE"
IN_LOGISTICS = f"{UNIT_ROOT}/IN_LOGISTICS"
OUT_EQUIPPED = f"{UNIT_ROOT}/OUT_EQUIPPED"

# Blender path
BLENDER_PATH = f"{DRIVE_ROOT}/EXODUS_AI_MODELS/blender-4.0.0-linux-x64/blender"

# Add codebase to path
import sys
sys.path.insert(0, CODEBASE)

print("╔════════════════════════════════════════════════════════════╗")
print("║           LOGISTICS DEPOT — PRODUCTION MODE                ║")
print("╚════════════════════════════════════════════════════════════╝")
print(f"\n✓ Drive Root: {DRIVE_ROOT}")
print(f"✓ Unit Root: {UNIT_ROOT}")

In [ ]:
# Vérifier/Installer Blender
import os
from pathlib import Path

if not Path(BLENDER_PATH).exists():
    print("⏳ Installation Blender 4.0...")
    !wget -q --show-progress https://download.blender.org/release/Blender4.0/blender-4.0.0-linux-x64.tar.xz
    !mkdir -p "{DRIVE_ROOT}/EXODUS_AI_MODELS"
    !tar -xf blender-4.0.0-linux-x64.tar.xz -C "{DRIVE_ROOT}/EXODUS_AI_MODELS/"
    !rm blender-4.0.0-linux-x64.tar.xz
    print("✓ Blender 4.0 installé")
else:
    print("✓ Blender 4.0 prêt")

# Créer dossiers si nécessaires
Path(IN_LOGISTICS).mkdir(parents=True, exist_ok=True)
Path(OUT_EQUIPPED).mkdir(parents=True, exist_ok=True)
Path(f"{IN_LOGISTICS}/props_library").mkdir(parents=True, exist_ok=True)

print("✓ Dossiers prêts")

## 🔍 Cellule 2: Scan Automatique du Dossier IN_LOGISTICS

In [ ]:
import json
from pathlib import Path
from datetime import datetime

def scan_inputs():
    """Scanne le dossier IN_LOGISTICS et identifie les fichiers à traiter."""
    
    inputs = {
        "actors": [],
        "production_plan": None,
        "props_library": []
    }
    
    in_path = Path(IN_LOGISTICS)
    
    # Chercher les .blend (actors animés)
    for blend_file in in_path.glob("*.blend"):
        inputs["actors"].append({
            "name": blend_file.stem,
            "path": str(blend_file),
            "size_mb": blend_file.stat().st_size / (1024 * 1024)
        })
    
    # Chercher le PRODUCTION_PLAN.JSON
    plan_path = in_path / "PRODUCTION_PLAN.JSON"
    if plan_path.exists():
        with open(plan_path, 'r') as f:
            plan = json.load(f)
        inputs["production_plan"] = {
            "path": str(plan_path),
            "scenes": len(plan.get("scenes", [])),
            "total_actions": sum(len(s.get("props_actions", [])) for s in plan.get("scenes", []))
        }
    
    # Chercher les props
    props_path = in_path / "props_library"
    if props_path.exists():
        for ext in [".glb", ".gltf", ".fbx", ".obj", ".blend"]:
            for prop_file in props_path.glob(f"*{ext}"):
                inputs["props_library"].append({
                    "id": prop_file.stem,
                    "path": str(prop_file),
                    "format": ext
                })
    
    return inputs

# Exécuter le scan
inputs = scan_inputs()

print("╔════════════════════════════════════════════════════════════╗")
print("║                    SCAN INPUTS                             ║")
print("╚════════════════════════════════════════════════════════════╝")

print(f"\n📁 Actors trouvés: {len(inputs['actors'])}")
for actor in inputs['actors']:
    print(f"   └── {actor['name']}.blend ({actor['size_mb']:.1f} MB)")

if inputs['production_plan']:
    print(f"\n📋 Production Plan: ✓")
    print(f"   └── Scenes: {inputs['production_plan']['scenes']}")
    print(f"   └── Actions: {inputs['production_plan']['total_actions']}")
else:
    print(f"\n📋 Production Plan: ✗ (PRODUCTION_PLAN.JSON requis!)")

print(f"\n🔧 Props Library: {len(inputs['props_library'])} props")
for prop in inputs['props_library'][:10]:
    print(f"   └── {prop['id']} ({prop['format']})")
if len(inputs['props_library']) > 10:
    print(f"   └── ... et {len(inputs['props_library']) - 10} autres")

## ⚙️ Cellule 3: Traitement Batch de Tous les Actors

In [ ]:
import subprocess
import time
from datetime import datetime

def process_actor(actor_path, plan_path, props_library, output_name):
    """Traite un actor avec le pipeline complet."""
    
    start_time = time.time()
    
    cmd = [
        "python", f"{CODEBASE}/EXO_02_LOGISTICS.py",
        "--drive-root", DRIVE_ROOT,
        "--actor-blend", actor_path,
        "--production-plan", plan_path,
        "--props-library", props_library,
        "--output-dir", OUT_EQUIPPED,
        "--output-name", output_name,
        "--blender-path", BLENDER_PATH,
        "-v"
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    elapsed = time.time() - start_time
    
    return {
        "success": result.returncode == 0,
        "elapsed_seconds": elapsed,
        "stdout": result.stdout,
        "stderr": result.stderr
    }

# Configuration batch
BATCH_CONFIG = {
    "process_all": True,  # True = tous les actors, False = liste spécifique
    "specific_actors": [],  # Liste des noms d'actors spécifiques si process_all=False
    "continue_on_error": True,  # Continuer même si un actor échoue
}

print("╔════════════════════════════════════════════════════════════╗")
print("║                 BATCH PROCESSING                           ║")
print("╚════════════════════════════════════════════════════════════╝")

In [ ]:
# Exécution du batch
from IPython.display import clear_output

if not inputs['production_plan']:
    print("❌ ERREUR: PRODUCTION_PLAN.JSON requis!")
    print("   Placez votre fichier dans IN_LOGISTICS/PRODUCTION_PLAN.JSON")
elif not inputs['actors']:
    print("❌ ERREUR: Aucun actor .blend trouvé!")
    print("   Placez vos fichiers .blend dans IN_LOGISTICS/")
else:
    # Préparer la liste des actors à traiter
    if BATCH_CONFIG['process_all']:
        actors_to_process = inputs['actors']
    else:
        actors_to_process = [a for a in inputs['actors'] if a['name'] in BATCH_CONFIG['specific_actors']]
    
    print(f"\n🎬 Actors à traiter: {len(actors_to_process)}")
    print(f"📋 Plan: {inputs['production_plan']['path']}")
    print(f"🔧 Props: {len(inputs['props_library'])} disponibles")
    print("\n" + "="*60)
    
    results = []
    batch_start = time.time()
    
    for i, actor in enumerate(actors_to_process, 1):
        print(f"\n[{i}/{len(actors_to_process)}] Processing: {actor['name']}")
        print("-" * 40)
        
        output_name = f"{actor['name']}_equipped"
        
        result = process_actor(
            actor['path'],
            inputs['production_plan']['path'],
            f"{IN_LOGISTICS}/props_library",
            output_name
        )
        
        result['actor'] = actor['name']
        result['output_name'] = output_name
        results.append(result)
        
        status = "✓" if result['success'] else "✗"
        print(f"{status} {actor['name']}: {result['elapsed_seconds']:.1f}s")
        
        if not result['success']:
            print(f"   Error: {result['stderr'][-500:] if result['stderr'] else 'Unknown'}")
            if not BATCH_CONFIG['continue_on_error']:
                print("\n❌ Batch interrompu (continue_on_error=False)")
                break
    
    batch_elapsed = time.time() - batch_start
    
    # Résumé
    print("\n" + "="*60)
    print("                    BATCH SUMMARY")
    print("="*60)
    
    successes = sum(1 for r in results if r['success'])
    failures = len(results) - successes
    
    print(f"\n✓ Succès: {successes}/{len(results)}")
    print(f"✗ Échecs: {failures}/{len(results)}")
    print(f"⏱ Temps total: {batch_elapsed:.1f}s")
    print(f"⏱ Moyenne par actor: {batch_elapsed/len(results):.1f}s")

## 📊 Cellule 4: Rapport de Production

In [ ]:
import json
from datetime import datetime

def generate_batch_report(results, inputs):
    """Génère un rapport de production complet."""
    
    report = {
        "timestamp": datetime.now().isoformat(),
        "unit": "02_LOGISTICS_DEPOT",
        "version": "1.0.0",
        "summary": {
            "total_actors": len(results),
            "successful": sum(1 for r in results if r['success']),
            "failed": sum(1 for r in results if not r['success']),
            "total_time_seconds": sum(r['elapsed_seconds'] for r in results),
            "props_available": len(inputs['props_library'])
        },
        "inputs": {
            "production_plan": inputs['production_plan'],
            "props_count": len(inputs['props_library'])
        },
        "actors": []
    }
    
    for result in results:
        actor_entry = {
            "name": result['actor'],
            "status": "SUCCESS" if result['success'] else "FAILED",
            "processing_time_seconds": result['elapsed_seconds'],
            "outputs": {}
        }
        
        if result['success']:
            abc_path = f"{OUT_EQUIPPED}/{result['output_name']}.abc"
            blend_path = f"{OUT_EQUIPPED}/{result['output_name']}.blend"
            
            if Path(abc_path).exists():
                actor_entry['outputs']['alembic'] = {
                    "path": abc_path,
                    "size_mb": Path(abc_path).stat().st_size / (1024 * 1024)
                }
            if Path(blend_path).exists():
                actor_entry['outputs']['blend'] = {
                    "path": blend_path,
                    "size_mb": Path(blend_path).stat().st_size / (1024 * 1024)
                }
        else:
            actor_entry['error'] = result['stderr'][-1000:] if result['stderr'] else "Unknown error"
        
        report['actors'].append(actor_entry)
    
    return report

# Générer le rapport si des résultats existent
if 'results' in dir() and results:
    report = generate_batch_report(results, inputs)
    
    # Sauvegarder le rapport
    report_path = f"{OUT_EQUIPPED}/batch_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(report_path, 'w') as f:
        json.dump(report, f, indent=2)
    
    print("╔════════════════════════════════════════════════════════════╗")
    print("║                 PRODUCTION REPORT                          ║")
    print("╚════════════════════════════════════════════════════════════╝")
    
    print(f"\n📊 Rapport sauvegardé: {report_path}")
    print(f"\n=== Résumé ===")
    print(f"  Total: {report['summary']['total_actors']} actors")
    print(f"  Succès: {report['summary']['successful']}")
    print(f"  Échecs: {report['summary']['failed']}")
    print(f"  Temps total: {report['summary']['total_time_seconds']:.1f}s")
    
    print(f"\n=== Outputs Générés ===")
    for actor in report['actors']:
        status = "✓" if actor['status'] == 'SUCCESS' else "✗"
        print(f"\n{status} {actor['name']}")
        if actor['status'] == 'SUCCESS' and actor['outputs']:
            for output_type, info in actor['outputs'].items():
                print(f"   └── {output_type}: {info['size_mb']:.1f} MB")
else:
    print("Aucun résultat de batch à rapporter.")
    print("Exécutez d'abord la Cellule 3.")

In [ ]:
# Lister tous les outputs générés
print("\n=== Contenu OUT_EQUIPPED ===")

out_path = Path(OUT_EQUIPPED)
if out_path.exists():
    files = list(out_path.iterdir())
    files.sort(key=lambda x: x.stat().st_mtime, reverse=True)
    
    total_size = 0
    for f in files:
        size = f.stat().st_size / (1024 * 1024)
        total_size += size
        icon = "📦" if f.suffix == ".abc" else "📄" if f.suffix == ".blend" else "📊" if f.suffix == ".json" else "📁"
        print(f"{icon} {f.name} ({size:.1f} MB)")
    
    print(f"\n📊 Total: {len(files)} fichiers, {total_size:.1f} MB")
else:
    print("Dossier OUT_EQUIPPED vide ou non créé")

---
## 🎯 Checklist Production

- [ ] Drive monté
- [ ] Blender 4.0 installé
- [ ] PRODUCTION_PLAN.JSON présent
- [ ] Props library remplie
- [ ] Actors .blend présents
- [ ] Batch exécuté
- [ ] Rapport généré
- [ ] Outputs vérifiés